# Chapter 11: Conditional Logic

There are use-cases where conditionals are requirement like an if-else.


In [2]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import Session
import pandas as pd


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Case Expressions

Instead of using joins, list of customer names whether it is an individual or a business


In [3]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            c.cust_id
            , c.fed_id
            , (CASE c.cust_type_cd
                WHEN 'I' THEN (
                    SELECT
                        CONCAT('i-', i.fname, ' ', i.lname)
                    FROM individual i
                    WHERE c.cust_id = i.cust_id
                )
                WHEN 'B' THEN (
                    SELECT
                        CONCAT('b-', b.name)
                    FROM business b
                    WHERE b.cust_id = c.cust_id
                )
                ELSE 'UNKNOWN'
                END) name
        FROM
            customer c
        ;
        """,
        con=session.connection()
    )

print(df)

    cust_id       fed_id                      name
0         1  111-11-1111            i-James Hadley
1         2  222-22-2222           i-Susan Tingley
2         3  333-33-3333            i-Frank Tucker
3         4  444-44-4444            i-John Hayward
4         5  555-55-5555         i-Charles Frasier
5         6  666-66-6666            i-John Spencer
6         7  777-77-7777          i-Margaret Young
7         8  888-88-8888             i-Louis Blake
8         9  999-99-9999          i-Richard Farley
9        10   04-1111111     b-Chilton Engineering
10       11   04-2222222  b-Northeast Cooling Inc.
11       12   04-3333333      b-Superior Auto Body
12       13   04-4444444      b-AAA Insurance Inc.


# Selective Aggregation

Find all account whose account balances don't agree with the raw transaction data.

In [4]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            CONCAT('ALERT! : Account #' , a.account_id, ' has incorrect balance') alert
        FROM
            account a
        WHERE (a.avail_balance, a.pending_balance) <> 
            (SELECT
                SUM(
                    CASE
                        WHEN t.funds_avail_date > CURRENT_TIMESTAMP()
                            THEN 0
                        WHEN t.txn_type_cd = 'DBT'
                            THEN -1 * t.amount
                        ELSE
                            t.amount
                    END
                ) avail
                , SUM(
                    IF(t.txn_type_cd = 'DBT', -1 * t.amount, t.amount)
                ) pend
            FROM transaction t
            WHERE t.account_id = a.account_id
        );
        """,
        con=session.connection()
    )

print(df)

                                         alert
0    ALERT! : Account #1 has incorrect balance
1    ALERT! : Account #2 has incorrect balance
2    ALERT! : Account #3 has incorrect balance
3    ALERT! : Account #4 has incorrect balance
4    ALERT! : Account #5 has incorrect balance
5    ALERT! : Account #7 has incorrect balance
6    ALERT! : Account #8 has incorrect balance
7   ALERT! : Account #10 has incorrect balance
8   ALERT! : Account #11 has incorrect balance
9   ALERT! : Account #12 has incorrect balance
10  ALERT! : Account #13 has incorrect balance
11  ALERT! : Account #14 has incorrect balance
12  ALERT! : Account #15 has incorrect balance
13  ALERT! : Account #17 has incorrect balance
14  ALERT! : Account #18 has incorrect balance
15  ALERT! : Account #19 has incorrect balance
16  ALERT! : Account #21 has incorrect balance
17  ALERT! : Account #22 has incorrect balance
18  ALERT! : Account #23 has incorrect balance
19  ALERT! : Account #24 has incorrect balance
20  ALERT! : 

# Checking for Existence

Check which customers have a checking and saving account.


In [5]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            c.cust_id
            , c.fed_id
            , c.cust_type_cd
            , (SELECT
                IF(
                    EXISTS (
                        SELECT 1
                        FROM account a
                        WHERE a.cust_id = c.cust_id
                          AND a.product_cd = 'CHK'
                    ),
                    'Y',
                    'N'
                )
            ) has_checking
            , (SELECT
                IF(
                    EXISTS (
                        SELECT 1
                        FROM account a
                        WHERE a.cust_id = c.cust_id
                          AND a.product_cd = 'SAV'
                    ),
                    'Y',
                    'N'
                )
            ) has_saving
        FROM customer c;
        """,
        con=session.connection()
    )

print(df)

    cust_id       fed_id cust_type_cd has_checking has_saving
0         1  111-11-1111            I            Y          Y
1         2  222-22-2222            I            Y          Y
2         3  333-33-3333            I            Y          N
3         4  444-44-4444            I            Y          Y
4         5  555-55-5555            I            Y          N
5         6  666-66-6666            I            Y          N
6         7  777-77-7777            I            N          N
7         8  888-88-8888            I            Y          Y
8         9  999-99-9999            I            Y          N
9        10   04-1111111            B            Y          N
10       11   04-2222222            B            N          N
11       12   04-3333333            B            Y          N
12       13   04-4444444            B            N          N


What about how many accounts does a customer have? Count up to three and use '3 or more' to express large numbers

In [6]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            c.cust_id
            , c.fed_id
            , c.cust_type_cd
            , (
                CASE (
                    SELECT
                        COUNT(*)
                    FROM account a
                    WHERE a.cust_id = c.cust_id
                )
                    WHEN 0 THEN 'None'
                    WHEN 1 THEN '1'
                    WHEN 2 THEN '2'
                    ELSE '3 or more'
                END
            ) num_accounts
        FROM customer c;
        """,
        con=session.connection()
    )

print(df)

    cust_id       fed_id cust_type_cd num_accounts
0         1  111-11-1111            I    3 or more
1         2  222-22-2222            I            2
2         3  333-33-3333            I            2
3         4  444-44-4444            I    3 or more
4         5  555-55-5555            I            1
5         6  666-66-6666            I            2
6         7  777-77-7777            I            1
7         8  888-88-8888            I            2
8         9  999-99-9999            I    3 or more
9        10   04-1111111            B            2
10       11   04-2222222            B            1
11       12   04-3333333            B            1
12       13   04-4444444            B            1


# Division by Zero Errors

Find the percentages that a customer divides all their assets across the bank products.

In [30]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            a.cust_id
            , a.product_cd
            , ROUND(a.avail_balance / (
                IF(prod_tots.tot_balance = 0, 1., prod_tots.tot_balance)
            ), 3) * 100. percent_of_total
        FROM account a JOIN (
            SELECT
                a2.product_cd
                , SUM(a2.avail_balance) tot_balance
            FROM
                account a2
            GROUP BY a2.product_cd
        ) prod_tots ON a.product_cd = prod_tots.product_cd
        ORDER BY a.product_cd, a.cust_id;
        """,
        con=session.connection()
    )

print(df)

    cust_id product_cd  percent_of_total
0        10        BUS               0.0
1        11        BUS             100.0
2         1         CD              15.4
3         6         CD              51.3
4         7         CD              25.6
5         9         CD               7.7
6         1        CHK               1.4
7         2        CHK               3.1
8         3        CHK               1.4
9         4        CHK               0.7
10        5        CHK               3.1
11        6        CHK               0.2
12        8        CHK               4.8
13        9        CHK               0.2
14       10        CHK              32.3
15       12        CHK              52.8
16        3         MM              13.0
17        4         MM              32.2
18        9         MM              54.8
19        1        SAV              26.9
20        2        SAV              10.8
21        4        SAV              41.4
22        8        SAV              20.9
23       13     

I am concerned about adding percentages correctly, so let's use ROLLUP to check the results

In [28]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            a.cust_id
            , a.product_cd
            , ROUND(a.avail_balance / (
                IF(prod_tots.tot_balance = 0, 1., prod_tots.tot_balance)
            ), 3) * 100. percent_of_total
            , SUM(
                a.avail_balance / (
                    IF(prod_tots.tot_balance = 0, 1., prod_tots.tot_balance)
                ) * 100.
            ) all_percentages
        FROM account a JOIN (
            SELECT
                a2.product_cd
                , SUM(a2.avail_balance) tot_balance
            FROM
                account a2
            GROUP BY a2.product_cd
        ) prod_tots ON a.product_cd = prod_tots.product_cd
        GROUP BY a.product_cd, a.cust_id, percent_of_total WITH ROLLUP
        ORDER BY a.product_cd, a.cust_id;
        """,
        con=session.connection()
    )

print(df)

    cust_id product_cd  percent_of_total  all_percentages
0       NaN       None               NaN       600.000000
1       NaN        BUS               NaN       100.000000
2      10.0        BUS               0.0         0.000000
3      10.0        BUS               NaN         0.000000
4      11.0        BUS               NaN       100.000000
5      11.0        BUS             100.0       100.000000
6       NaN         CD               NaN       100.000000
7       1.0         CD              15.4        15.384615
8       1.0         CD               NaN        15.384615
9       6.0         CD              51.3        51.282051
10      6.0         CD               NaN        51.282051
11      7.0         CD              25.6        25.641026
12      7.0         CD               NaN        25.641026
13      9.0         CD               7.7         7.692308
14      9.0         CD               NaN         7.692308
15      NaN        CHK               NaN       100.000000
16      1.0   

# Exercises

## 11-1
Rewrite the following query so that the same results are achieved with as few WHEN clauses as possible

```mysql
SELECT
    emp_id
    , CASE title
        WHEN 'President' THEN 'Management'
        WHEN 'Vice President' THEN 'Management'
        WHEN 'Treasurer' THEN 'Management'
        WHEN 'Loan Manager' THEN 'Management'
        WHEN 'Operations Manager' THEN 'Operations'
        WHEN 'Head Teller' THEN 'Operations'
        WHEN 'Teller' THEN 'Operations'
        ELSE 'Unknown'
    END
FROM
    employee;
```

## 11-2
Rewrite the following query so that the result set contains a single row with four columns (one for each branch). Name the four columns 'branch_1' through 'branch_4'

```mysql
SELECT open_branch_id, COUNT(*)
FROM account
GROUP BY open_branch_id;
```

| open_branch_id | COUNT(*) |
|:---------------|:---------|
| 1              | 8        |
| 2              | 7        |
| 3              | 3        |
| 4              | 6        |

In [42]:
# 11-1

with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            emp_id
            , CASE
                WHEN title IN ('President', 'Vice President', 'Treasurer', 'Loan Manager') THEN 'Management'
                WHEN title IN ('Operations Manager', 'Head Teller', 'Teller') THEN 'Operations'
                ELSE 'Unknown'
            END super_title
        FROM
            employee;
        """,
        con=session.connection()
    )

print(df)

    emp_id super_title
0        1  Management
1        2  Management
2        3  Management
3        4  Operations
4        5  Management
5        6  Operations
6        7  Operations
7        8  Operations
8        9  Operations
9       10  Operations
10      11  Operations
11      12  Operations
12      13  Operations
13      14  Operations
14      15  Operations
15      16  Operations
16      17  Operations
17      18  Operations


In [40]:
# 11-2

with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            SUM(CASE WHEN open_branch_id = 1 THEN 1 ELSE 0 END) branch_1
            , SUM(CASE WHEN open_branch_id = 2 THEN 1 ELSE 0 END) branch_2
            , SUM(CASE WHEN open_branch_id = 3 THEN 1 ELSE 0 END) branch_3
            , SUM(CASE WHEN open_branch_id = 4 THEN 1 ELSE 0 END) branch_4
        FROM
            account;
        """,
        con=session.connection()
    )

print(df)

   branch_1  branch_2  branch_3  branch_4
0       8.0       7.0       3.0       6.0
